In [1]:
from BigDFT import Datasets as D, Calculators as C, Inputfiles as I, Logfiles as lf
from BigDFT.Database import Molecules
import sys,os
sys.path.insert(0,'../')
import StatPol as SP

basepath = os.getcwd()
basepath

'/home/dalessandro/work/Statical_polarizability/Calculations'

In [2]:
omp = 2
mpi_run = 'mpirun -np 8'

# Workflow

This notebook contains the complete workflow to extract the statical polarizability of the HG dataset.

## Construction of the relevant tools

Define the functions needed to perform the workflow

In [3]:
# Service functions 

def change_workdir(molecule,study,datadir):
    """
    Ensure that the path Data_directory/molecule exists and
    move there. 
    """
    from futile.Utils import ensure_dir
    for d in [datadir,molecule]:
        ensure_dir(d)
        os.chdir(d)
    path='-'.join(study)
    ensure_dir(path)
    return '-'.join([molecule,path]),path

def get_psp(molecule):
    """
    Check if the all the atoms of molecule have the psp of the type nlss_aw and nlcc_ss. 
    If it is true the corresponding pseudo is added to the list of psp to be performed, otherwise 
    only hgh_k is included.
    """
    from os import path as p
    import Routines as R
    possible_psp=['nlcc_aw','nlcc_ss']
    to_do=[True,True]
    for at in R.get_atoms(molecule):
        psp_available = [p.isfile(p.join('psp_'+psp,'psppar.'+at)) for psp in possible_psp]
        to_do = [ a and b for a,b in zip(to_do,psp_available)]
    return ['hgh_k'] + [psp for yes,psp in zip(to_do,possible_psp) if yes]

def split_dataset(dataset,length=10):
    """
    Split a list of molecules into a list of list with length elements. 
    """
    splitted = [[] for num in range(int(len(dataset)/length)+1)]
    for ind,mol in enumerate(dataset):
        splitted[int(ind/length)].append(mol)
    return splitted

In [4]:
# Tools for the ground state

def get_converged_input_energy(dataset,rtol,atol):
    data=dataset.seek_convergence(atol=atol,rtol=rtol,attribute='energy')
    return dataset.runs[dataset.names.index(D.name_from_id(data[0]))]['input'],data[1]

def find_gs_domain(label,rtol,atol,path,input,posinp,code):
    """
    Use the seek_convergence method of Dataset to perform a convergence procedure on
    the rmult value for a gs computation.
    """
    crmult=input['dft']['rmult'][0]
    rmult_fine=input['dft']['rmult'][1]
    rmult_list=map(float,range(int(crmult),11))
    seek_for_rmult = D.Dataset(label=label+'(GS)',run_dir=path,posinp=posinp)
    for rm in rmult_list:
        input.set_rmult(coarse=rm,fine=rmult_fine)
        seek_for_rmult.append_run(id={'rmult':rm},runner=code,input=input)
    input_gs,log_gs=get_converged_input_energy(seek_for_rmult,rtol,atol)
    return {'input_gs':input_gs, 'dataset_gs': seek_for_rmult,'log_gs':log_gs}

def gs_study(molecule,study,code,options):
    """"
    Workflow for the convergence analysis of the ground state.
    
    Args : 
       molecule (str) : name of the molecule
       study (touple) : the couple (xc,psp)
       code (runner)  : instance of SystemCalculator
       options (dict) : dictionary with the computational options
    """
    import Routines as R
    hgrids=options.get('hgrids')
    rmult_fine=options.get('rmult_fine')
    wf_convergence=options.get('wf_convergence')
    crmult_start=options.get('crmult',4.0)
    rtol=options.get('rtol_gs',10*wf_convergence)
    atol=options.get('atol_gs',1.e-3)
    reference_results=options.get('reference_data')
    datadir=options.get('data_directory','Data')

    initial_dir=os.path.abspath(os.path.dirname('.'))
    label,path=change_workdir(molecule,study,datadir)
    
    posinp=Molecules.Molecule(molecule)
    
    inp = I.Inputfile()
    inp.set_hgrid(hgrids)
    inp.set_rmult(coarse=crmult_start,fine=rmult_fine)
    R.set_xc[study](inp,molecule,datadir)
    inp.set_wavefunction_convergence(wf_convergence)
    mpol=int(reference_results[molecule]['mpol_ref'])-1   
    if mpol >0: inp.spin_polarize(mpol)
    data= find_gs_domain(label,10*wf_convergence,atol,path,inp,posinp,code)
    os.chdir(initial_dir)
    
    data['molecule']=molecule
    data['posinp']=posinp
    data['study']=study
    return data 

In [5]:
# Tools for alpha
def extract_statical_polarizability(label,rtol,atol,path,input_for_alpha,posinp,code,ref):
    
    # seek for the field convergence
    ints=[1e-2,5e-3,1e-3,5e-4,1e-4]
    alpha_field=D.Dataset(label=label+'(field)',run_dir=path)
    for f in ints:
        alpha_field.append_run(id={'f':f},runner=SP.build_alpha_dataset(input=input_for_alpha,posinp=posinp,run_dir=path,runner=code,intensity=f))
    try:
        data_field=alpha_field.seek_convergence(rtol=rtol,atol=atol)
        intensity=data_field[0]['f']
    except:
        print ('Convergence in field not reached')
        intensity=ints[-1]
    
    # seek for the rmult convergence
    rmult_fine=input_for_alpha['dft']['rmult'][1]
    rmult_list=map(float,range(int(input_for_alpha['dft']['rmult'][0]),11))
    alpha_rmult=D.Dataset(label=label+'(domain)',run_dir=path)
    for rm in rmult_list:
        input_for_alpha.set_rmult(coarse=rm,fine=rmult_fine)
        alpha_rmult.append_run(id={'f':intensity,'rmult':rm},runner=SP.build_alpha_dataset(input=input_for_alpha,posinp=posinp,run_dir=path,runner=code,intensity=intensity))
    try:
        import numpy as np
        data_final=alpha_rmult.seek_convergence(rtol=rtol,atol=atol)
        AuToA = 0.5291772085**3
        alpha_ref_ii = np.array(ref)/AuToA
        eps = 100.0 * (data_final[1]-alpha_ref_ii)/alpha_ref_ii
        print ('Relative difference in %',eps.tolist())
    except LookupError as e:
        print ('Convergence in domain not reached',e)
        data_final=None
    return {'alpha_convergence': data_final, 'dataset_field': alpha_field, 'dataset_rmult': alpha_rmult}


def alpha_study(gs_data,code,options):
    """"
    Workflow for the convergence analysis for the computation of alpha.
    
    Args : 
       gs_data        : output of gs_study
       code (runner)  : instance of SystemCalculator
       options (dict) : dictionary with the computational options
    """
    import copy
    xc_conversion={'lda_pw': 'lda-SPW92', 'pbe': 'pbe', 'pbe0': 'pbe0'}
    rtol=options.get('rtol',1.e-2)
    atol=options.get('atol',1.e-3)
    reference_results=options.get('reference_data')
    datadir=options.get('data_directory','Data')
    study=gs_data['study']
    molecule=gs_data['molecule']
    posinp=gs_data['posinp']
    xc=study[0]
    ref_data=reference_results[molecule][xc_conversion[xc]]
    
    initial_dir=os.path.abspath(os.path.dirname('.'))
    label,path=change_workdir(molecule,study,datadir)
    input_gs=copy.deepcopy(gs_data['input_gs'])
        
    data=extract_statical_polarizability(label,rtol,atol,path,input_gs,posinp,code,ref=ref_data)
    os.chdir(initial_dir)
    
    return data

In [6]:
def data_to_save(data,options):
    """
    Define the dictionary to be saved on file for data analysis
    """
    from copy import deepcopy
    res = {}
    for s,values in data.iteritems():
        res[s] = {}
        if not (values['alpha_convergence'] is None):
            res[s]['alpha_convergence'] = (values['alpha_convergence'][0],values['alpha_convergence'][1].tolist())
        else : 
            res[s]['alpha_convergence'] = values['alpha_convergence']
        
        res[s]['posinp'] = {}
        pos = values['posinp']
        for key in pos:
            res[s]['posinp'][key] = pos[key]
        
        res[s]['input_gs'] = {}
        inp = values['input_gs']
        for key in inp:
            res[s]['input_gs'][key] = inp[key]
        
    save_options = deepcopy(options)
    save_options.pop('reference_data')
    res['options'] = save_options
    return res

## Single study analysis

Build the dataset using the HG data written in hg_data.yaml. To each molecule of the dataset associate a set of studies=(xc,psp)

In [8]:
import yaml
HG_data=yaml.load(open('../HG Dataset/hg_data.yaml'))

In [9]:
code=C.SystemCalculator(omp=omp,mpi_run=mpi_run,skip=True,verbose=False)

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -np 8 /home/dalessandro/Applications/BigDFT/binaries/v1.8.4/install/bin/bigdft


In [10]:
options={'wf_convergence':1.e-6,'hgrids':0.3,'rmult_fine':9.0,'rtol':1.e-3,'atol':1.e-3,\
         'reference_data':HG_data,'data_directory':'Data'}

We can perform a single study with this workflow

In [11]:
molecule = 'CO'
xc = 'pbe'
psp = 'nlcc_aw'
study_data = {}

In [12]:
gs_data=gs_study(molecule,(xc,psp),code,options)

Fetching results for id " {'rmult': 4.0} "
Fetching results for id " {'rmult': 5.0} "
Fetching results for id " {'rmult': 6.0} "
Convergence reached in Dataset "CO-pbe-nlcc_aw(GS)" for id " {'rmult': 5.0} "


In [13]:
gs_data

{'dataset_gs': <BigDFT.Datasets.Dataset instance at 0x7fc3f8122c68>,
 'input_gs': {'dft': {'gnrm_cv': 1e-06,
   'hgrids': 0.3,
   'ixc': 11,
   'rmult': [5.0, 9.0]}},
 'log_gs': <BigDFT.Logfiles.Logfile instance at 0x7fc3f854dc20>,
 'molecule': 'CO',
 'posinp': {'positions': [{'C': [0.0, 0.0, 0.0], 'frag': ['molecule', '0']},
   {'O': [0.0, 0.0, 1.1282], 'frag': ['molecule', '0']}],
  'units': 'angstroem'},
 'study': ('pbe', 'nlcc_aw')}

In [14]:
study_data[(molecule,xc,psp)]=alpha_study(gs_data,code,options)
study_data[(molecule,xc,psp)].update(gs_data)

Fetching results for id " {'f': 0.01} "
Fetching results for id " {'f': 0.005} "
Fetching results for id " {'f': 0.001} "
Convergence reached in Dataset "CO-pbe-nlcc_aw(field)" for id " {'f': 0.005} "
Fetching results for id " {'rmult': 5.0, 'f': 0.005} "
Fetching results for id " {'rmult': 6.0, 'f': 0.005} "
Fetching results for id " {'rmult': 7.0, 'f': 0.005} "
Fetching results for id " {'rmult': 8.0, 'f': 0.005} "
Convergence reached in Dataset "CO-pbe-nlcc_aw(domain)" for id " {'rmult': 7.0, 'f': 0.005} "
('Relative difference in %', [0.1266037733590598, 0.1266037733590598, -0.35506403390880337])


In [15]:
study_data

{('CO',
  'pbe',
  'nlcc_aw'): {'alpha_convergence': ({'f': 0.005, 'rmult': 7.0},
   array([12.540766, 12.540766, 15.889695])), 'dataset_field': <BigDFT.Datasets.Dataset instance at 0x7fc3f82a05f0>, 'dataset_gs': <BigDFT.Datasets.Dataset instance at 0x7fc3f8122c68>, 'dataset_rmult': <BigDFT.Datasets.Dataset instance at 0x7fc3f81285f0>, 'input_gs': {'dft': {'gnrm_cv': 1e-06,
    'hgrids': 0.3,
    'ixc': 11,
    'rmult': [5.0,
     9.0]}}, 'log_gs': <BigDFT.Logfiles.Logfile instance at 0x7fc3f854dc20>, 'molecule': 'CO', 'posinp': {'positions': [{'C': [0.0,
      0.0,
      0.0],
     'frag': ['molecule', '0']},
    {'O': [0.0, 0.0, 1.1282], 'frag': ['molecule', '0']}],
   'units': 'angstroem'}, 'study': ('pbe', 'nlcc_aw')}}

To check if convergence has been reached we can look at 'alpha_convergence'. 

If the value of 'f' is the last value of the field intensity list provided in the extract_statical_polarizability function, then the field convergence has not been reached. Moreover, if 'alpha_convergence' is None the rmult convergence on alpha has failed. 

In [16]:
data_to_save(study_data,options)

{'options': {'atol': 0.001,
  'data_directory': 'Data',
  'hgrids': 0.3,
  'rmult_fine': 9.0,
  'rtol': 0.001,
  'wf_convergence': 1e-06},
 ('CO',
  'pbe',
  'nlcc_aw'): {'alpha_convergence': ({'f': 0.005, 'rmult': 7.0},
   [12.540765999999998,
    12.540765999999998,
    15.889695000000001]), 'input_gs': {'dft': {'gnrm_cv': 1e-06,
    'hgrids': 0.3,
    'ixc': 11,
    'rmult': [5.0, 9.0]}}, 'posinp': {'positions': [{'C': [0.0, 0.0, 0.0],
     'frag': ['molecule', '0']},
    {'O': [0.0, 0.0, 1.1282], 'frag': ['molecule', '0']}],
   'units': 'angstroem'}}}

## Dataset analysis

In the same way we can run the whole dataset (or a part of it). The results of the dataset (processed by data_to_save) are stored in the full_data dictionary. This dictionary can be dumped in a Yaml file and updated with new computatitions so that the complete dataset can be runned in more than one step. 

In [24]:
import yaml
HG_data=yaml.load(open('../HG Dataset/hg_data.yaml'))
# set the study for H to sp
HG_data['H']['spin_pol'] = 'sp'

In [25]:
# for computational reason it can be useful to split the complete dataset into the sp and the nsp ones.
# Moreover, each dataset can be divided into groups. 
sp_dataset = []
nsp_dataset = []
for mol in HG_data:
    if HG_data[mol]['spin_pol'] == 'nsp': 
        nsp_dataset.append(mol)
    else : 
        sp_dataset.append(mol)

# remove Li2 because the gs calculation does not converge with the
# actual range of parameters
ind =  sp_dataset.index('Li2')
del sp_dataset[ind]

sp_dataset.sort()
nsp_dataset.sort()
sp_split = split_dataset(sp_dataset)
nsp_split = split_dataset(nsp_dataset)

In [26]:
print len(nsp_dataset)
print len(sp_dataset)

74
57


In [27]:
sp_split

[['BH2', 'BN', 'BO', 'BS', 'Be', 'BeH', 'C2H', 'C2H3', 'CH2-t', 'CH2F'],
 ['CH2NH', 'CH2PH', 'CH3', 'CH3O', 'CN', 'F2', 'FCO', 'FH-OH', 'H', 'H2CN'],
 ['H2O-Li', 'HCHS', 'HCO', 'HCP', 'HNO', 'HNS', 'HO2', 'HOF', 'Li', 'N'],
 ['N2H2', 'NCO', 'NH', 'NH2', 'NO', 'NOCl', 'NP', 'Na', 'Na2', 'NaLi'],
 ['O2', 'O3', 'OCl', 'OF', 'OF2', 'OH', 'P', 'P2', 'PH', 'PH2'],
 ['PS', 'S2', 'SCl', 'SF', 'SH', 'SO-trip', 'SiH3']]

We also build a complete dataset that can be run to write all the results on file

In [28]:
full_dataset = []
for mol in HG_data:
        full_dataset.append(mol)
# remove Li2 because the gs calculation does not converge with the
# actual range of parameters
ind =  full_dataset.index('Li2')
del full_dataset[ind]
full_dataset.sort()

In [29]:
len(full_dataset)

131

In [31]:
# check if some molecule folder is missing in Data
for mol in full_dataset:
    d = os.path.join('Data',mol)
    if not os.path.isdir(d):
        print mol

FH-OH
H2O-Li


In [32]:
full_dataset_split = split_dataset(full_dataset)

In [33]:
for sets in full_dataset_split:
    print sets

['AlF', 'Ar', 'BF', 'BH2', 'BH2Cl', 'BH2F', 'BH3', 'BHF2', 'BN', 'BO']
['BS', 'Be', 'BeH', 'BeH2', 'C2H', 'C2H2', 'C2H3', 'C2H4', 'CH2-t', 'CH2BH']
['CH2F', 'CH2NH', 'CH2PH', 'CH3', 'CH3BH2', 'CH3Cl', 'CH3F', 'CH3NH2', 'CH3O', 'CH3OH']
['CH3SH', 'CH4', 'CN', 'CO', 'CO2', 'CS', 'CSO', 'Cl2', 'ClCN', 'ClF']
['F2', 'FCN', 'FCO', 'FH-OH', 'FNO', 'H', 'H2', 'H2CN', 'H2O', 'H2O-Li']
['HBO', 'HBS', 'HCCCl', 'HCCF', 'HCHO', 'HCHS', 'HCN', 'HCO', 'HCONH2', 'HCOOH']
['HCP', 'HCl', 'HF', 'HNC', 'HNO', 'HNS', 'HO2', 'HOCl', 'HOF', 'HOOH']
['He', 'Li', 'LiBH4', 'LiCN', 'LiCl', 'LiH', 'Mg', 'Mg2', 'N', 'N2']
['N2H2', 'N2H4', 'NCO', 'NH', 'NH2', 'NH2Cl', 'NH2F', 'NH2OH', 'NH3', 'NH3O']
['NO', 'NOCl', 'NP', 'Na', 'Na2', 'NaCN', 'NaCl', 'NaH', 'NaLi', 'Ne']
['O2', 'O3', 'OCl', 'OCl2', 'OF', 'OF2', 'OH', 'P', 'P2', 'P2H4']
['PH', 'PH2', 'PH2OH', 'PH3', 'PH3O', 'PS', 'S2', 'S2H2', 'SCl', 'SCl2']
['SF', 'SF2', 'SH', 'SH2', 'SO-trip', 'SO2', 'SiH3', 'SiH3Cl', 'SiH3F', 'SiH4']
['SiO']


Run the dataset

In [34]:
options={'wf_convergence':1.e-6,'hgrids':0.3,'rmult_fine':9.0,'rtol':1.e-3,'atol':1.e-3,\
         'reference_data':HG_data,'data_directory':'Data'}

In [35]:
code=C.SystemCalculator(omp=omp,mpi_run=mpi_run,skip=True,verbose=False)

Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -np 8 /home/dalessandro/Applications/BigDFT/binaries/v1.8.4/install/bin/bigdft


In [ ]:
full_data = {}
for molecule in ['H2O-Li']:          #full_dataset: #full_dataset_split[2]+full_dataset_split[3]:
    for xc in ['lda_pw','pbe','pbe0']:
        psp = ['hgh_k'] if xc != 'pbe' else get_psp(molecule)
        for p in psp:
            print ''
            print molecule,xc,p
            gs_data=gs_study(molecule,(xc,p),code,options)
            full_data[(molecule,xc,p)]=alpha_study(gs_data,code,options)
            full_data[(molecule,xc,p)].update(gs_data)


H2O-Li lda_pw hgh_k
Fetching results for id " {'rmult': 4.0} "
Fetching results for id " {'rmult': 5.0} "
Fetching results for id " {'rmult': 6.0} "
Fetching results for id " {'rmult': 7.0} "
Convergence reached in Dataset "H2O-Li-lda_pw-hgh_k(GS)" for id " {'rmult': 6.0} "
Fetching results for id " {'f': 0.01} "
Fetching results for id " {'f': 0.005} "
Fetching results for id " {'f': 0.001} "
Fetching results for id " {'f': 0.0005} "
Convergence reached in Dataset "H2O-Li-lda_pw-hgh_k(field)" for id " {'f': 0.001} "
Fetching results for id " {'rmult': 6.0, 'f': 0.001} "
Fetching results for id " {'rmult': 7.0, 'f': 0.001} "


In [41]:
#full_data

In [ ]:
# CH3NH2 pbe0 hgh_k is trying to perform the field convergence for pbe0 hgh_k....why?

In [61]:
results = data_to_save(full_data,options)
#results

In [57]:
#full_results = {}
full_results = yaml.load(open('full_results.yaml'))

In [62]:
full_results['CH3NH2','pbe0','hgh_k']

{'alpha_convergence': ({'f': 0.005, 'rmult': 8.0},
  [28.570789, 24.91872, 24.14379]),
 'input_gs': {'dft': {'gnrm_cv': 1e-06,
   'hgrids': 0.3,
   'ixc': 'PBE0',
   'rmult': [5.0, 9.0]},
  'psppar.C': {'Pseudopotential XC': 11},
  'psppar.H': {'Pseudopotential XC': 11},
  'psppar.N': {'Pseudopotential XC': 11}},
 'posinp': {'positions': [{'N': [0.7505020558, -0.1263257432, -1.4e-09],
    'frag': ['molecule', '0']},
   {'H': [1.1371866572, 0.3468207227, -0.80589993], 'frag': ['molecule', '0']},
   {'H': [1.1371866572, 0.3468207227, 0.80589993], 'frag': ['molecule', '0']},
   {'C': [-0.7051562913, 0.018043978, -1.4e-09], 'frag': ['molecule', '0']},
   {'H': [-1.1126045962, -0.4830900647, 0.8751002462],
    'frag': ['molecule', '0']},
   {'H': [-1.1126045962, -0.4830900647, -0.8751002462],
    'frag': ['molecule', '0']},
   {'H': [-1.071740754, 1.0485550259, 0.0], 'frag': ['molecule', '0']}],
  'units': 'angstroem'}}

In [54]:
full_results.update(results)

In [37]:
#full_results.keys()

In [56]:
with open('full_results.yaml', 'w') as outfile:
    yaml.dump(full_results, outfile, default_flow_style=False)